<hr>

## Data Loads for KMeans Clustering Demo ##
- **demo_ofs.UK_Retail_Data**


In [1]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Load demo_ofs.UK_Retail_Data  data
# /* -------------------------------------------------------- */

import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE Demo_ofs.UK_Retail_Data')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    
    qry = '''
CREATE MULTISET TABLE demo_ofs.UK_Retail_Data, 
    STORAGE = TD_OFSSTORAGE
     (
      InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
      StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
      Description VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
      Quantity BIGINT,
      InvoiceDate TIMESTAMP(6),
      UnitPrice FLOAT,
      CustomerID FLOAT,
      Country VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC);
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.UK_Retail_Data 
    SELECT InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/UK_RETAIL_DATA/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    
    cur.execute(qry)